In [1]:
%%capture _ 
# ^ this just silences warnings
import torch
import mlflow
import hiddenlayer as HL

from model.collectdata_mdsA import collect_data
from model.alt_loss_A import Loss
from model.training import trainNet, select_gpu
from model.utilities import load_full_state, count_parameters, Params, save_to_mlflow

from model.autoencoder_models import UNet
from model.autoencoder_models import UNetPlusPlus

In [2]:
args = Params(
    batch_size=64,
    device = select_gpu(0),
    epochs=30,
    lr=4e-4,
    experiment_name='UNet', 
    asymmetry_parameter=2.5
)

1 available GPUs (initially using device 0):
  0 GeForce RTX 2080 Ti


In [3]:
train_loader = collect_data(
    '/share/lazy/sokoloff/ML-data_A/Aug14_80K_train.h5',
#     '/share/lazy/sokoloff/ML-data_AA/Oct03_80K_train.h5',
#     '/share/lazy/sokoloff/ML-data_AA/Oct03_40K_train.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_1.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_3.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_4.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_5.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_6.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_7.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_8.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_9.h5',
    batch_size=args['batch_size'],
    masking=True,
    shuffle=False,
    load_XandXsq=False,
#     device = args['device'], 
    load_xy=False
)

val_loader = collect_data(
    '/share/lazy/sokoloff/ML-data_AA/Oct03_20K_val.h5',
    batch_size=args['batch_size'],
    slice=slice(256 * 39),
    masking=True, 
    shuffle=False,
    load_XandXsq=False,
    load_xy=False)


Loading data...
Loaded /share/lazy/sokoloff/ML-data_A/Aug14_80K_train.h5 in 11.53 s
Constructing 80000 event dataset took 0.2034 s
Loading data...
Loaded /share/lazy/sokoloff/ML-data_AA/Oct03_20K_val.h5 in 2.533 s
Constructing 9984 event dataset took 0.04051 s


In [4]:
mlflow.tracking.set_tracking_uri('file:/share/lazy/pv-finder_model_repo')
mlflow.set_experiment(args['experiment_name'])

Traceback (most recent call last):
  File "/apps/miniconda3/envs/june2020-gpu/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/apps/miniconda3/envs/june2020-gpu/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/apps/miniconda3/envs/june2020-gpu/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/share/lazy/pv-finder_model_repo/ML/meta.yaml' does not exist.


In [5]:
model = UNet().to(args['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss = Loss(epsilon=1e-5,coefficient=args['asymmetry_parameter'])

#print(model.i1.size())

In [ ]:
# load_full_state(model, optimizer, '/share/lazy/pv-finder_model_repo/0/a868d4b8ec0642b39a7156f3dd894dfb/artifacts/run_stats.pyt', freeze_weights=False)

run_name = 'u-net'

# tune kernel based on gpu
#torch.backends.cudnn.benchmark=True
train_iter = enumerate(trainNet(model, optimizer, loss, train_loader, val_loader, args['epochs'], notebook=True))
with mlflow.start_run(run_name = run_name) as run:
    for i, result in train_iter:
        save_to_mlflow({
            'Metric: Training loss':result.cost,
            'Metric: Validation loss':result.val,
            'Metric: Efficiency':result.eff_val.eff_rate,
            'Metric: False positive rate':result.eff_val.fp_rate,
            'Param: Asymmetry':args['asymmetry_parameter'],
            'Param: Epochs':args['epochs'],
        }, step=i)

Number of batches: train = 1250, val = 156


/data/home/michael24peters/pv-finder_fork/pv-finder/notebooks/model/training.py:94: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  file=sys.stderr,


Epoch 0: train=18.9447, val=5.55068, took 108.38 s
  Validation Found 50352 of 54504, added 1567 (eff 92.38%) (0.157 FP/event)


Epoch 1: train=3.20394, val=5.56606, took 110.24 s
  Validation Found 50597 of 54504, added 1601 (eff 92.83%) (0.16 FP/event)


Epoch 2: train=3.13078, val=5.56825, took 109.36 s
  Validation Found 50578 of 54504, added 1403 (eff 92.80%) (0.141 FP/event)


...

Epoch 27: train=2.87508, val=5.35836, took 110.7 s
  Validation Found 51213 of 54504, added 1624 (eff 93.96%) (0.163 FP/event)


Epoch 28: train=2.87237, val=5.39042, took 108.02 s
  Validation Found 51118 of 54504, added 1531 (eff 93.79%) (0.153 FP/event)


Epoch 29: train=2.86551, val=5.38016, took 110.23 s
  Validation Found 51389 of 54504, added 1753 (eff 94.28%) (0.176 FP/event)


Epoch 30: train=2.86034, val=5.40226, took 109.81 s
  Validation Found 51253 of 54504, added 1540 (eff 94.04%) (0.154 FP/event)


Epoch 31: train=2.85826, val=5.39768, took 110.95 s
  Validation Found 51302 of 54504, added 1662 (eff 94.13%) (0.166 FP/event)
